# Vector Arithmetic: Testing Cross-Lingual Compositionality

**Goal:** Test if semantic vector arithmetic works in causal geometry across languages.

**Hypothesis:** If `'vier' - 'four'` captures "German-ness", then:
```
'vier' - 'four' + 'planet' ≈ German word for planet
```

**Best case:** We get 'Planet' or German astronomy terms

**Why this matters:** The linear representation hypothesis predicts that semantic relationships should be compositional—we can add and subtract meaning vectors. If this works across languages in causal geometry, it's strong evidence that the model learned language-independent semantic directions.

**Method:**
1. Compute synthetic vector: `v = 'vier' - 'four' + 'planet'`
2. Find nearest neighbors by causal distance
3. Find nearest neighbors by causal cosine similarity
4. Decode and analyze results

**Inputs:**
- Token embeddings from 07.57
- Known tokens: 'four' (34024), 'planet' (50074)
- Target token: 'vier' (57093) from German

## Configuration

In [10]:
# Model
MODEL_NAME = 'Qwen/Qwen3-4B-Instruct-2507'

# Data paths
DISTANCES_PATH = '../data/vectors/distances_causal_32000_full.npy'
METADATA_PATH = '../data/vectors/distances_causal_32000.pt'
METRIC_TENSOR_PATH = '../data/vectors/causal_metric_tensor_qwen3_4b.pt'

# Tokens for arithmetic
TOKENS = {
    'four': 34024,      # English number
    'planet': 50074,    # English astronomy
    'vier': 57093,      # German number (from 07.57 analysis)
}

# Analysis parameters
TOP_N = 20  # How many neighbors to show

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Arithmetic: 'vier' - 'four' + 'planet' = ?")
print(f"  Top N results: {TOP_N}")

Configuration:
  Model: Qwen/Qwen3-4B-Instruct-2507
  Arithmetic: 'vier' - 'four' + 'planet' = ?
  Top N results: 20


## Setup

In [11]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt

print("✓ Imports complete")

✓ Imports complete


## Load Data

In [12]:
print("Loading data...\n")

# Metadata and token indices
print(f"Loading metadata from {METADATA_PATH}...")
metadata = torch.load(METADATA_PATH, weights_only=False)
token_indices = metadata['token_indices'].numpy()
token_to_idx = {tid: idx for idx, tid in enumerate(token_indices)}

# Tokenizer
print(f"\nLoading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Model (for embeddings)
print(f"\nLoading model (for unembedding matrix)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map='cpu',
)
gamma = model.lm_head.weight.data.to(torch.float32).cpu()
sampled_embeddings = gamma[token_indices]

# Metric tensor
print(f"\nLoading causal metric tensor from {METRIC_TENSOR_PATH}...")
metric_data = torch.load(METRIC_TENSOR_PATH, weights_only=False)
M = metric_data['M'].to(torch.float32).cpu()

print(f"\n✓ All data loaded")
print(f"  N tokens in sample: {len(token_indices):,}")
print(f"  Embedding dimension: {sampled_embeddings.shape[1]}")

Loading data...

Loading metadata from ../data/vectors/distances_causal_32000.pt...

Loading tokenizer from Qwen/Qwen3-4B-Instruct-2507...

Loading model (for unembedding matrix)...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Loading causal metric tensor from ../data/vectors/causal_metric_tensor_qwen3_4b.pt...

✓ All data loaded
  N tokens in sample: 32,000
  Embedding dimension: 2560


## Verify Tokens Are In Sample

In [13]:
print("Checking if required tokens are in our sample...\n")

all_present = True
for name, token_id in TOKENS.items():
    if token_id in token_to_idx:
        idx = token_to_idx[token_id]
        text = tokenizer.decode([token_id])
        print(f"✓ '{name}' (token {token_id}, index {idx}): '{text}'")
    else:
        print(f"❌ '{name}' (token {token_id}) NOT IN SAMPLE")
        all_present = False

if not all_present:
    raise ValueError("Some required tokens are not in the sample!")

print("\n✓ All required tokens present")

Checking if required tokens are in our sample...

✓ 'four' (token 34024, index 27606): 'four'
✓ 'planet' (token 50074, index 20043): 'planet'
✓ 'vier' (token 57093, index 16750): ' vier'

✓ All required tokens present


## Compute Synthetic Vector: 'vier' - 'four' + 'planet'

In [14]:
print("Computing synthetic vector...\n")

# Get embeddings
idx_vier = token_to_idx[TOKENS['vier']]
idx_four = token_to_idx[TOKENS['four']]
idx_planet = token_to_idx[TOKENS['planet']]

v_vier = sampled_embeddings[idx_vier]
v_four = sampled_embeddings[idx_four]
v_planet = sampled_embeddings[idx_planet]

# Arithmetic
v_synthetic = v_vier - v_four + v_planet

print(f"v_synthetic = 'vier' - 'four' + 'planet'")
print(f"\nSynthetic vector properties:")
print(f"  Shape: {v_synthetic.shape}")
print(f"  Euclidean norm: {torch.norm(v_synthetic).item():.2f}")

# Compute causal norm of synthetic vector
causal_norm_synthetic = torch.sqrt(v_synthetic @ M @ v_synthetic).item()
print(f"  Causal norm: {causal_norm_synthetic:.2f} logometers")

Computing synthetic vector...

v_synthetic = 'vier' - 'four' + 'planet'

Synthetic vector properties:
  Shape: torch.Size([2560])
  Euclidean norm: 1.62
  Causal norm: 79.53 logometers


## Find Nearest Neighbors by Causal Distance

In [15]:
print("\nComputing causal distances to all tokens...\n")

# Compute causal distances: d_M(u, v) = sqrt((u-v)^T M (u-v))
N = len(token_indices)
causal_distances = np.zeros(N)

for i in range(N):
    diff = sampled_embeddings[i] - v_synthetic
    causal_distances[i] = torch.sqrt(diff @ M @ diff).item()

# Sort by distance (ascending)
sorted_indices = np.argsort(causal_distances)
top_indices = sorted_indices[:TOP_N]

print(f"Top {TOP_N} tokens by CAUSAL DISTANCE:")
print(f"{'Rank':<6} {'Distance':<12} {'Token ID':<10} {'Text':<40}")
print("=" * 80)

for rank, idx in enumerate(top_indices, 1):
    dist = causal_distances[idx]
    tid = token_indices[idx]
    text = tokenizer.decode([tid])
    print(f"{rank:<6} {dist:<12.2f} {tid:<10} {text:<40}")

print("\n" + "=" * 80)


Computing causal distances to all tokens...

Top 20 tokens by CAUSAL DISTANCE:
Rank   Distance     Token ID   Text                                    
1      54.55        50074      planet                                  
2      70.09        59634      Planet                                  
3      74.09        57093       vier                                   
4      76.59        150187     ㋩                                       
5      76.64        122768     䓫                                       
6      76.66        149829     ﭵ                                       
7      76.67        150219     ㋯                                       
8      76.67        146965     ꡒ                                       
9      76.67        151036     쩻                                       
10     76.68        151114     흟                                       
11     76.69        149731     꾈                                       
12     76.69        149798     큉                        

## Find Nearest Neighbors by Causal Cosine Similarity

In [16]:
print("\nComputing causal cosine similarities to all tokens...\n")

# Compute causal cosine: cos(θ_M) = (u^T M v) / (||u||_M · ||v||_M)
causal_dot_products = sampled_embeddings @ M @ v_synthetic  # [N]

# Compute causal norms for all tokens
causal_norms = torch.sqrt(torch.sum(sampled_embeddings @ M * sampled_embeddings, dim=1)).numpy()

# Cosine similarities
causal_cosines = (causal_dot_products / (torch.tensor(causal_norms) * causal_norm_synthetic)).numpy()

# Sort by cosine (descending)
sorted_indices_cos = np.argsort(-causal_cosines)
top_indices_cos = sorted_indices_cos[:TOP_N]

print(f"Top {TOP_N} tokens by CAUSAL COSINE SIMILARITY:")
print(f"{'Rank':<6} {'Cosine':<12} {'Distance':<12} {'Token ID':<10} {'Text':<40}")
print("=" * 80)

for rank, idx in enumerate(top_indices_cos, 1):
    cos_sim = causal_cosines[idx]
    dist = causal_distances[idx]
    tid = token_indices[idx]
    text = tokenizer.decode([tid])
    print(f"{rank:<6} {cos_sim:<12.4f} {dist:<12.2f} {tid:<10} {text:<40}")

print("\n" + "=" * 80)


Computing causal cosine similarities to all tokens...

Top 20 tokens by CAUSAL COSINE SIMILARITY:
Rank   Cosine       Distance     Token ID   Text                                    
1      0.7277       54.55        50074      planet                                  
2      0.5110       70.09        59634      Planet                                  
3      0.4429       74.09        57093       vier                                   
4      0.2699       76.59        150187     ㋩                                       
5      0.2673       76.66        149829     ﭵ                                       
6      0.2671       76.64        122768     䓫                                       
7      0.2667       76.67        150219     ㋯                                       
8      0.2658       76.67        146965     ꡒ                                       
9      0.2657       76.68        151114     흟                                       
10     0.2656       76.67        151036     쩻      

## Check for 'Planet' Specifically

In [17]:
# Token ID for 'Planet' (capitalized, from 07.57)
PLANET_CAPITALIZED = 59634

if PLANET_CAPITALIZED in token_to_idx:
    idx_Planet = token_to_idx[PLANET_CAPITALIZED]
    
    dist_to_Planet = causal_distances[idx_Planet]
    cos_to_Planet = causal_cosines[idx_Planet]
    
    # Find rank
    rank_by_distance = np.where(sorted_indices == idx_Planet)[0][0] + 1
    rank_by_cosine = np.where(sorted_indices_cos == idx_Planet)[0][0] + 1
    
    print("\n" + "=" * 80)
    print("CHECKING FOR 'Planet' (token 59634):")
    print("=" * 80)
    print(f"\nCausal distance: {dist_to_Planet:.2f} logometers (rank {rank_by_distance})")
    print(f"Causal cosine: {cos_to_Planet:.4f} (rank {rank_by_cosine})")
    
    if rank_by_distance <= 10:
        print(f"\n🎉 'Planet' is in the TOP 10 by causal distance!")
    elif rank_by_distance <= 20:
        print(f"\n✓ 'Planet' is in the top 20 by causal distance")
    else:
        print(f"\n'Planet' is rank {rank_by_distance} by causal distance")
    
    if rank_by_cosine <= 10:
        print(f"🎉 'Planet' is in the TOP 10 by causal cosine!")
    elif rank_by_cosine <= 20:
        print(f"✓ 'Planet' is in the top 20 by causal cosine")
    else:
        print(f"'Planet' is rank {rank_by_cosine} by causal cosine")
else:
    print("\n⚠️  Token 'Planet' (59634) not in our sample")


CHECKING FOR 'Planet' (token 59634):

Causal distance: 70.09 logometers (rank 2)
Causal cosine: 0.5110 (rank 2)

🎉 'Planet' is in the TOP 10 by causal distance!
🎉 'Planet' is in the TOP 10 by causal cosine!


## Overlap Analysis

In [18]:
print("\n" + "=" * 80)
print("OVERLAP ANALYSIS")
print("=" * 80)

# Find tokens that appear in BOTH top N lists
top_by_distance = set(token_indices[top_indices])
top_by_cosine = set(token_indices[top_indices_cos])

overlap = top_by_distance & top_by_cosine

print(f"\nTokens in top {TOP_N} by BOTH metrics:")
print(f"  Count: {len(overlap)}")

if overlap:
    print(f"\nTokens appearing in both lists:")
    for tid in sorted(overlap):
        text = tokenizer.decode([tid])
        idx = token_to_idx[tid]
        dist = causal_distances[idx]
        cos = causal_cosines[idx]
        print(f"  • {tid}: '{text}' (distance={dist:.2f}, cosine={cos:.4f})")


OVERLAP ANALYSIS

Tokens in top 20 by BOTH metrics:
  Count: 20

Tokens appearing in both lists:
  • 50074: 'planet' (distance=54.55, cosine=0.7277)
  • 57093: ' vier' (distance=74.09, cosine=0.4429)
  • 59634: 'Planet' (distance=70.09, cosine=0.5110)
  • 79269: ' ForCanBeConverted' (distance=76.70, cosine=0.2648)
  • 122768: '䓫' (distance=76.64, cosine=0.2671)
  • 123072: '�' (distance=76.69, cosine=0.2649)
  • 146965: 'ꡒ' (distance=76.67, cosine=0.2658)
  • 149413: '칕' (distance=76.69, cosine=0.2651)
  • 149577: 'ᗭ' (distance=76.69, cosine=0.2650)
  • 149731: '꾈' (distance=76.69, cosine=0.2651)
  • 149798: '큉' (distance=76.69, cosine=0.2651)
  • 149829: 'ﭵ' (distance=76.66, cosine=0.2673)
  • 150187: '㋩' (distance=76.59, cosine=0.2699)
  • 150219: '㋯' (distance=76.67, cosine=0.2667)
  • 150308: '쓻' (distance=76.69, cosine=0.2649)
  • 150737: '⭞' (distance=76.69, cosine=0.2650)
  • 150877: '냵' (distance=76.69, cosine=0.2651)
  • 151036: '쩻' (distance=76.67, cosine=0.2656)
  • 151114:

## Interpretation

**What we're looking for:**

1. **Best case:** 'Planet' (German capitalized) appears in top results
   - Would confirm: `'vier' - 'four' + 'planet' ≈ 'Planet'`
   - Evidence: Language shift vector works compositionally

2. **Good case:** German words appear in top results (even if not astronomy-related)
   - Would suggest: `'vier' - 'four'` captures some "German-ness"
   - But: Language vector might not transfer perfectly across semantic domains

3. **Moderate case:** Astronomy words appear (but not necessarily German)
   - Would suggest: 'planet' dominates the arithmetic
   - Language shift vector too weak or context-dependent

4. **Worst case:** Random tokens or neither German nor astronomy
   - Would suggest: Vector arithmetic doesn't compose in this space
   - Linear representation hypothesis fails for cross-lingual transfer

**Key metrics:**
- If 'Planet' is in top 10 by distance → **strong evidence for compositionality**
- If 'Planet' is in top 20 → **moderate evidence**
- If 'Planet' appears in both top-20 lists → **very strong evidence**

---

## Results Analysis

**What we found:**
- 'Planet' is **#2** by both causal distance (70.09 logometers) and causal cosine (0.51)
- This is **farther** than the original distance from 'planet' to 'Planet' (43.56 logometers, cosine 0.71)

**Conclusion:** The unscaled arithmetic `'vier' - 'four' + 'planet'` moved us **away** from 'Planet', not toward it.

**Question:** What if we need to **scale** the language shift vector?

## Testing Scaled Vector Arithmetic

**Hypothesis:** Maybe we need to scale the language shift vector:

```
v_Planet ≈ v_planet + α * (v_vier - v_four)
```

**Method:** Solve for the optimal α via least-squares projection:

```
α = (v_Planet - v_planet) · (v_vier - v_four) / ||v_vier - v_four||²
```

Then test if the scaled version gets us closer to 'Planet'.

In [19]:
print("\n" + "=" * 80)
print("TESTING SCALED VECTOR ARITHMETIC")
print("=" * 80)

# Get 'Planet' embedding
idx_Planet = token_to_idx[PLANET_CAPITALIZED]
v_Planet = sampled_embeddings[idx_Planet]

# Compute the language shift vector
v_shift = v_vier - v_four

# Compute the target displacement
v_target_displacement = v_Planet - v_planet

# Solve for α via least-squares projection
# α = (target · shift) / (shift · shift)
numerator = torch.dot(v_target_displacement, v_shift).item()
denominator = torch.dot(v_shift, v_shift).item()
alpha_optimal = numerator / denominator

print(f"\nOptimal scaling factor α = {alpha_optimal:.4f}")
print(f"\nThis means: v_Planet ≈ v_planet + {alpha_optimal:.4f} * (v_vier - v_four)")

# Construct the scaled synthetic vector
v_scaled = v_planet + alpha_optimal * v_shift

print(f"\nScaled synthetic vector properties:")
print(f"  Euclidean norm: {torch.norm(v_scaled).item():.2f}")

causal_norm_scaled = torch.sqrt(v_scaled @ M @ v_scaled).item()
print(f"  Causal norm: {causal_norm_scaled:.2f} logometers")

# Compute distance from scaled vector to 'Planet'
diff_to_Planet = v_scaled - v_Planet
dist_scaled_to_Planet = torch.sqrt(diff_to_Planet @ M @ diff_to_Planet).item()

# Compute cosine similarity
causal_norm_Planet = causal_norms[idx_Planet]
causal_dot_scaled_Planet = (v_scaled @ M @ v_Planet).item()
cos_scaled_to_Planet = causal_dot_scaled_Planet / (causal_norm_scaled * causal_norm_Planet)

print(f"\n{'=' * 80}")
print(f"COMPARISON: Unscaled vs Scaled vs Original")
print(f"{'=' * 80}")

print(f"\n1. Original 'planet' to 'Planet':")
print(f"   Distance: 43.56 logometers")
print(f"   Cosine: 0.71")

print(f"\n2. Unscaled synthetic (α=1) to 'Planet':")
print(f"   Distance: 70.09 logometers")
print(f"   Cosine: 0.51")

print(f"\n3. SCALED synthetic (α={alpha_optimal:.4f}) to 'Planet':")
print(f"   Distance: {dist_scaled_to_Planet:.2f} logometers")
print(f"   Cosine: {cos_scaled_to_Planet:.4f}")

print(f"\n{'=' * 80}")

# Determine outcome
if dist_scaled_to_Planet < 43.56:
    improvement_dist = 43.56 - dist_scaled_to_Planet
    print(f"✅ SCALED VERSION IS CLOSER by {improvement_dist:.2f} logometers!")
elif dist_scaled_to_Planet < 70.09:
    improvement_dist = 70.09 - dist_scaled_to_Planet
    print(f"✓ Scaled version is better than unscaled by {improvement_dist:.2f} logometers")
    print(f"  But still farther than original 'planet' → 'Planet'")
else:
    print(f"❌ Scaled version is no improvement")

if cos_scaled_to_Planet > 0.71:
    improvement_cos = cos_scaled_to_Planet - 0.71
    print(f"✅ SCALED VERSION IS MORE ALIGNED by {improvement_cos:.4f}!")
elif cos_scaled_to_Planet > 0.51:
    improvement_cos = cos_scaled_to_Planet - 0.51
    print(f"✓ Scaled version is better than unscaled by {improvement_cos:.4f}")
    print(f"  But still less aligned than original 'planet' → 'Planet'")
else:
    print(f"❌ Scaled version is no improvement in alignment")


TESTING SCALED VECTOR ARITHMETIC

Optimal scaling factor α = -0.0004

This means: v_Planet ≈ v_planet + -0.0004 * (v_vier - v_four)

Scaled synthetic vector properties:
  Euclidean norm: 1.17
  Causal norm: 58.35 logometers

COMPARISON: Unscaled vs Scaled vs Original

1. Original 'planet' to 'Planet':
   Distance: 43.56 logometers
   Cosine: 0.71

2. Unscaled synthetic (α=1) to 'Planet':
   Distance: 70.09 logometers
   Cosine: 0.51

3. SCALED synthetic (α=-0.0004) to 'Planet':
   Distance: 43.55 logometers
   Cosine: 0.7110

✅ SCALED VERSION IS CLOSER by 0.01 logometers!
✅ SCALED VERSION IS MORE ALIGNED by 0.0010!
